In [ ]:
import numpy as np
states = 20
pf = 0.4
V = np.zeros(states+1)
V[states] = 1

In [ ]:
for i in range(10000):
    for j in range(1,states):
        actions = range(1,min(j,states-j)+1)
        max_val = -1*np.inf
        for a in actions:
            temp = pf*V[j+a] + (1-pf)*V[j-a]
            if temp >= max_val:
                max_val = temp
        V[j] = max_val

In [ ]:
V

In [ ]:
policy = -1*np.ones(states+1)
for j in range(1,states):
    actions = range(1,min(j,states-j)+1)
    max_val = -1*np.inf
    for a in actions:
        temp = pf*V[j+a] + (1-pf)*V[j-a]
        print("(",j,",",a,")",":",temp)
        if temp >= max_val:
            max_val = temp
            policy[j] = a
    print("*******************")
#     V[j] = max_val

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1,states),V[1:states])

In [ ]:
#Monte-Carlo Control
Q = np.random.random((states+1,states+1))
returns = {}
for i in range(10000): #Number of episodes
    done = False
    episode = []
    s = np.random.randint(9) + 1
    while not done:
        if np.random.random() < 0.1:
            a = np.random.randint(min(s,states-s))+1
        else:
            actions = range(1,min(s,states-s)+1)
            a = np.argmax(Q[s,1:min(s,states-s)+1])
            a = actions[a]
            
        if np.random.random() < pf:
            ns = s+a
        else:
            ns = s-a
                        
        if ns == states:
            reward = 1
            done = True
        elif ns == 0:
            reward = 0
            done = True
        else:
            reward = 0
            done = False
            
        episode.append((s,a,reward,ns))
        s = ns
    
    total_reward = 0 
    for l in reversed(episode):
#         print(l)
        index = (l[0],l[1])
        if index not in returns.keys():
            returns[index] = []
        total_reward += l[2]
        returns[index].append(total_reward)
        
    for i in list(returns.keys()):
#         print(i[0],i[1])
        Q[i[0],i[1]] = np.average(returns[i])
#         print(Q[i[0],i[1]])
    
        
    
            
        
        

In [ ]:
policy_MC = -1*np.ones(states+1)
for s in range(1,states):
    actions = range(1,min(s,states-s)+1)
    a = np.argmax(Q[s,1:min(s,states-s)+1])
    policy_MC[s] = actions[a]

In [ ]:
policy_MC[1:10]

In [ ]:
#Off-Policy Monte-Carlo Control
Q = np.zeros((states+1,states+1))
C = np.zeros((states+1,states+1))
returns = {}
for i in range(100000): #Number of episodes
    done = False
    episode = []
    s = np.random.randint(9) + 1
    while not done:
        a = np.random.randint(min(s,states-s))+1          
        if np.random.random() < pf:
            ns = s+a
        else:
            ns = s-a
                        
        if ns == states:
            reward = 1
            done = True
        elif ns == 0:
            reward = 0
            done = True
        else:
            reward = 0
            done = False
            
        episode.append((s,a,reward,ns))
        s = ns
    
    total_reward = 0 
    W = 1
    for l in reversed(episode):
#         print(l)
        total_reward += l[2]
        C[l[0],l[1]] += W
        Q[l[0],l[1]] += (W/C[l[0],l[1]])*(total_reward - Q[l[0],l[1]])
        
        actions = range(1,min(l[0],states-l[0])+1)
        a = np.argmax(Q[l[0],1:min(l[0],states-l[0])+1])
        
        if actions[a] != l[1]:
            break
        W = W*(1/(1/len(actions)))
#     print("******************")

In [ ]:
policy_MC_off = -1*np.ones(states+1)
for s in range(1,states):
    actions = range(1,min(s,states-s)+1)
    a = np.argmax(Q[s,1:min(s,states-s)+1])
    policy_MC_off[s] = actions[a]

In [ ]:
policy_MC_off[1:10]

In [ ]:
#Q-Learning
Q = np.zeros((states+1,states+1))
count = np.zeros((states+1,states+1))
for i in range(10000): #Number of episodes
    done = False
    s = np.random.randint(9) + 1
    while not done:
        if np.random.random() < 0.2:
            a = np.random.randint(min(s,states-s))+1
        else:
            actions = range(1,min(s,states-s)+1)
            a = np.argmax(Q[s,1:min(s,states-s)+1])
            a = actions[a]
            
        ns = get_next_state(s,a)
                        
        if ns == states:
            reward = 1
            done = True
        elif ns == 0:
            reward = 0
            done = True
        else:
            reward = 0
            done = False
            
        if ns == 0 or ns == states:
            next_reward = 0
        else:
            next_reward = np.max(Q[ns,1:min(ns,states-ns)+1])
        
        count[s,a] +=1
        alpha = 1/count[s,a]
        
        Q[s,a] = (1-alpha)*Q[s,a] + alpha*(reward + next_reward)
        
        s = ns
    
        
    
            
        
        

In [ ]:
#environment 
def get_next_state(s,a):
    if np.random.random() < pf:
        ns = s+a
    else:
        ns = s-a
    return ns

In [ ]:
policy_Q = -1*np.ones(states+1)
for s in range(1,states):
    actions = range(1,min(s,states-s)+1)
    a = np.argmax(Q[s,1:min(s,states-s)+1])
    policy_Q[s] = actions[a]

In [ ]:
policy_Q[1:states]